# 6.6. File I/O

Vide exercício 2.

Até agora, discutimos como processar dados e como construir, treinar e testar modelos de aprendizado profundo. No entanto, em algum momento, esperamos estar felizes o suficiente com os modelos aprendidos que desejaremos salvar os resultados (finais ou parciais (checkpointing)) para uso posterior em vários contextos.

Portanto, é hora de aprender como carregar e armazenar vetores de peso individuais e modelos inteiros.

In [3]:
import torch
from torch import nn                        # para usar nn. ao invés de torch.nn
from torch.nn import functional as F        # para usar F. ao invés de torch.nn.functional

# https://pytorch.org/docs/stable/nn.functional.html

# 6.6.1. Loading and Saving Tensors

- Loading: torch.load
- Saving:  torch.save


Para tensores individuais, podemos invocar diretamente as funções __load__ e __save__ para lê-los e escrevê-los, respectivamente. Ambas as funções exigem que forneçamos um nome e save exigem como entrada a variável a ser salva.

In [6]:
x = torch.arange(4)
torch.save(x, 'x-file')   # x foi salvo na pasta raiz com o nome 'x-file'

# Pasta raiz: 
# G:\Meu Drive\0-DOUTORADO\0-Python\PycharmProjects\Deep_Learning

Agora podemos ler os dados do arquivo armazenado de volta na memória.

In [7]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

Podemos armazenar uma lista de tensores e lê-los de volta na memória.

In [13]:
y = torch.zeros(4)                # tensor y: tensor([0., 0., 0., 0.])
torch.save([x, y],'x-files')      # salva lista de tensores: [x,y]
x2, y2 = torch.load('x-files')    # carrega tensores salvos. x2 será igual ao elemento [0] e y2 será igual ao elemento [1].
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [16]:
x2

tensor([0, 1, 2, 3])

In [17]:
y2

tensor([0., 0., 0., 0.])

In [14]:
a = torch.load('x-files')        # Neste caso a lista inteira é carregada na variável a. 

In [15]:
a

[tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.])]

Podemos até escrever e ler um dicionário que mapeia de strings para tensores. Isso é conveniente quando queremos ler ou escrever todos os pesos em um modelo.

In [18]:
mydict = {'x': x, 'y': y}

In [19]:
mydict

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [20]:
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

# 6.6.2. Loading and Saving Model Parameters

A estrutura de aprendizado profundo fornece funcionalidades integradas para carregar e salvar redes inteiras.

Um detalhe importante a ser observado é que isso salva parâmetros do modelo e não o modelo inteiro. 

Vamos começar com nosso MLP familiar.

In [227]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [228]:
net

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [229]:
Y.shape

torch.Size([2, 10])

Em seguida, armazenamos os parâmetros do modelo como um arquivo com o nome “mlp.params”.

In [231]:
# armazenamos os parâmetros do modelo como um arquivo com o nome “mlp.params”
torch.save(net.state_dict(), 'mlp.params')         #  EXTENSÃO PARA PARÂMETROS: .params           

In [188]:
# Parâmetros:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.1637, -0.2161,  0.0046,  ...,  0.0528, -0.0274,  0.1120],
                      [ 0.0064,  0.1223,  0.0137,  ...,  0.0890,  0.0146, -0.0184],
                      [-0.1966,  0.0962,  0.0600,  ..., -0.1850, -0.0472, -0.2086],
                      ...,
                      [ 0.2130,  0.1504,  0.0305,  ..., -0.0195,  0.1584,  0.1350],
                      [-0.1411, -0.0099, -0.0943,  ..., -0.1738,  0.2106,  0.0988],
                      [ 0.2201,  0.0375,  0.0749,  ..., -0.1360,  0.1319, -0.0959]])),
             ('hidden.bias',
              tensor([ 0.2097, -0.1271,  0.0918, -0.0254, -0.0600, -0.0233, -0.0498, -0.1440,
                      -0.0146,  0.0687,  0.2167, -0.1991,  0.0698,  0.1813,  0.0134,  0.0491,
                       0.0433, -0.1657, -0.0784,  0.0357, -0.2121,  0.1998,  0.1318, -0.0928,
                       0.0950, -0.1568,  0.0545, -0.2223,  0.1367,  0.0169,  0.1443,  0.0868,
                       0.1441,

In [189]:
# nome e shape dos parâmetros:
[(name, param.shape) for name, param in net.named_parameters()]

[('hidden.weight', torch.Size([256, 20])),
 ('hidden.bias', torch.Size([256])),
 ('output.weight', torch.Size([10, 256])),
 ('output.bias', torch.Size([10]))]

Para recuperar o modelo, instanciamos um clone do modelo MLP original. Em vez de inicializar aleatoriamente os parâmetros do modelo, lemos os parâmetros armazenados no arquivo diretamente.

In [190]:
clone = MLP()                                      #  clone do modelo MLP original.      
clone.load_state_dict(torch.load('mlp.params'))    # carrega os dados salvos

<All keys matched successfully>

In [191]:
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

Como ambas as instâncias têm os mesmos parâmetros de modelo, o resultado computacional para a mesma entrada X deve ser o mesmo. Vamos verificar isso.

In [192]:
clone

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [193]:
Y_clone = clone(X)                        # Saída Y utilizando como entrada X
Y_clone

tensor([[ 0.0712,  0.1360,  0.5060, -0.0326,  0.3252,  0.4594, -0.0410,  0.6288,
          0.2902, -0.0191],
        [-0.0205,  0.0549,  0.0193,  0.1864,  0.2478,  0.1678,  0.0109,  0.1173,
          0.0301, -0.2687]], grad_fn=<AddmmBackward0>)

In [194]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [195]:
[(name, param.shape) for name, param in clone.named_parameters()]

[('hidden.weight', torch.Size([256, 20])),
 ('hidden.bias', torch.Size([256])),
 ('output.weight', torch.Size([10, 256])),
 ('output.bias', torch.Size([10]))]

In [196]:
clone.state_dict()['hidden.weight'] == net.state_dict()['hidden.weight']

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [197]:
clone.state_dict()['hidden.bias'] == net.state_dict()['hidden.bias']

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [198]:
clone.state_dict()['output.weight'] == net.state_dict()['output.weight']

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [199]:
clone.state_dict()['output.bias'] == net.state_dict()['output.bias']

tensor([True, True, True, True, True, True, True, True, True, True])

# 6.6.3. Resumo

- As funções save e load podem ser usadas para executar E/S de arquivo para objetos tensor. 

- Podemos salvar e carregar todos os conjuntos de parâmetros para uma rede por meio de um dicionário de parâmetros. 

- Salvar a arquitetura tem que ser feito em código, em vez de parâmetros.

In [232]:
# SAVE:
torch.save(net.state_dict(), 'mlp.params')   # salva todos os parâmetros do modelo.
 #  EXTENSÃO PARA PARÂMETROS: .params   

In [233]:
# LOAD:
# Recuperar/carregar os parâmetros salvos.
clone = MLP()                                    # instanciamos um clone do modelo MLP original
clone.load_state_dict(torch.load('mlp.params'))  # lemos os parâmetros armazenados no arquivo diretamente.
clone.eval()
 #  EXTENSÃO PARA PARÂMETROS: .params   

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [56]:
clone

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

# 6.6.4. Exercícios

# Ex.1 - Mesmo que não haja necessidade de implantar modelos treinados em um dispositivo diferente, quais são os benefícios práticos de armazenar parâmetros de modelo?

- Economia de tempo e recursos, utilizando parâmetros já direcionados/sintonizados (tuning) em uma direção correta.
- Evita retreinamento do modelo toda vez que for utilizá-lo.
- Criação de versão de modelos.
- Tranferência de parâmetros / Transferência de conhecimentos / Compartilhamento e Colaboração.

# Ex.2 - Suponha que queremos reutilizar apenas partes de uma rede para serem incorporadas a uma rede com uma arquitetura diferente. Como você faria para usar, digamos, as duas primeiras camadas de uma rede anterior em uma nova rede?

PARTE 1: CRIAÇÃO E ANÁLISE DA REDE.

In [202]:
import torch
from torch import nn
from d2l import torch as d2l
from collections import OrderedDict

In [203]:
# CLASSE PARA REDE SEM ENTRADAS (LazyLinear)
class MLP(d2l.Classifier):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
                                 nn.LazyLinear(50),
                                 nn.ReLU(),
                                 nn.LazyLinear(20),
                                 nn.ReLU(),
                                 nn.LazyLinear(2)
                                 )

In [204]:
net = MLP()       # Rede sem entradas

In [205]:
net               # Observe: LazyLinear(in_features=0

MLP(
  (net): Sequential(
    (0): LazyLinear(in_features=0, out_features=50, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=0, out_features=20, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=0, out_features=2, bias=True)
  )
)

In [206]:
X = torch.rand(size=(10, 100))    # Dados de entrada X

In [207]:
Y = net(X)                        # Saída Y utilizando como entrada X
Y

tensor([[ 0.1661, -0.1892],
        [ 0.1658, -0.2083],
        [ 0.1912, -0.1713],
        [ 0.1990, -0.2247],
        [ 0.1873, -0.1955],
        [ 0.1901, -0.1793],
        [ 0.1880, -0.1796],
        [ 0.1754, -0.1882],
        [ 0.1913, -0.2157],
        [ 0.1692, -0.2003]], grad_fn=<AddmmBackward0>)

In [208]:
net                            # Observe: Linear(in_features=100)

MLP(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

- Observe que agora net alterou-se de LazyLinear para Linear.
- Antes: in_features=0
- Agora: in_features=100

Logo, a entrada X foi aplicada.

net é uma rede com 5 camadas: 0 a 4.

In [209]:
torch.save(net.state_dict(), 'mlp.params')

In [210]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (net): Sequential(
    (0): LazyLinear(in_features=0, out_features=50, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=0, out_features=20, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=0, out_features=2, bias=True)
  )
)

In [213]:
clone

MLP(
  (net): Sequential(
    (0): LazyLinear(in_features=0, out_features=50, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=0, out_features=20, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=0, out_features=2, bias=True)
  )
)

In [215]:
# Acessando apenas parte da rede:
reused_layer = clone.net[:2]
reused_layer

Sequential(
  (0): LazyLinear(in_features=0, out_features=50, bias=True)
  (1): ReLU()
)

------------------

# PARTE 2: SALVANDO AS DUAS PRIMEIRAS CAMADAS (0) E (2):

Para isto será utilizado o OrderedDict, pois está no mesmo formato de net.state_dict().

type(net.state_dict())  \
out: collections.OrderedDict

In [171]:
# VISUALIZANDO AS CAMADAS:
[(name, param.shape) for name, param in net.named_parameters()]

[('net.0.weight', torch.Size([50, 100])),
 ('net.0.bias', torch.Size([50])),
 ('net.2.weight', torch.Size([20, 50])),
 ('net.2.bias', torch.Size([20])),
 ('net.4.weight', torch.Size([2, 20])),
 ('net.4.bias', torch.Size([2]))]

In [139]:
type(net.state_dict())     # .state_dict() é um OrderedDict:6

# Its API is essentially the same as dict . However, OrderedDict iterates over keys and values in the same order that the keys were inserted. 

collections.OrderedDict

In [140]:
# torch.save(net.state_dict(), 'mlp.params')   # salva todos os parâmetros do modelo.

from collections import OrderedDict

# OrderedDict:
camadas = OrderedDict([
    ('net.0.weight' ,net.state_dict()['net.0.weight']),
    ('net.0.bias' ,net.state_dict()['net.0.bias']),
    ('net.2.weight' ,net.state_dict()['net.2.weight']),
    ('net.2.bias' ,net.state_dict()['net.2.bias'])             ])
# camadas é um OrderedDict:4

torch.save(camadas, 'mlp.params')                                     # salva camadas
 


LOAD:  \
Recuperar/carregar os parâmetros salvos.  \
O código abaixo apresenta erro pois ``camadas`` possui apenas 4 dos 6 parâmetros. Logo, é preciso adicionar os parâmetros restantes. 

clone = MLP()                                    # instanciamos um clone do modelo MLP original  \
clone.load_state_dict(torch.load('mlp.params'))  # lemos os parâmetros armazenados no arquivo diretamente.  \
clone.eval()  

_RuntimeError: Error(s) in loading state_dict for MLP:_  \
_Missing key(s) in state_dict: "net.4.weight", "net.4.bias"._ 

Faltam:
'net.4.weight', 
'net.4.bias'.

In [147]:
# DECLARANDO OS PARÂMETROS FALTANTES E INCORPORANDO-OS AO OrderedDict CAMADAS:

 # ('net.4.weight', torch.Size([2, 20])),
 # ('net.4.bias', torch.Size([2]))]

camadas['net.4.weight']  = torch.randn(2, 20) 
camadas['net.4.bias']    = torch.randn(2) 

In [148]:
camadas

OrderedDict([('net.0.weight',
              tensor([[ 0.0719,  0.0439,  0.0277,  ...,  0.0840, -0.0769,  0.0545],
                      [ 0.0731, -0.0967, -0.0125,  ..., -0.0629,  0.0282,  0.0599],
                      [ 0.0411, -0.0464,  0.0003,  ..., -0.0458, -0.0645, -0.0345],
                      ...,
                      [-0.0678, -0.0042, -0.0967,  ...,  0.0188,  0.0742, -0.0594],
                      [ 0.0163,  0.0156,  0.0225,  ..., -0.0875, -0.0565,  0.0687],
                      [ 0.0847, -0.0123,  0.0827,  ..., -0.0834,  0.0106,  0.0904]])),
             ('net.0.bias',
              tensor([-0.0533, -0.0312, -0.0219,  0.0977,  0.0210,  0.0183,  0.0334, -0.0884,
                      -0.0443, -0.0231, -0.0813,  0.0199, -0.0857,  0.0860, -0.0488, -0.0392,
                       0.0963, -0.0159, -0.0127, -0.0698,  0.0716,  0.0124, -0.0793,  0.0905,
                      -0.0093, -0.0768,  0.0497,  0.0229, -0.0832,  0.0245,  0.0223,  0.0136,
                       0.0872, -

Logo, as duas primeiras camadas ((0) e (2)) são da rede anterior e a última camada ((4)) foi criada posteriormente. Desta forma, posso utilizar estas camadas na rede anterior ou em uma nova rede.

In [149]:
# utilizando na rede anterior:
clone = MLP()                                    # instanciamos um clone do modelo MLP original  \
# clone.load_state_dict(torch.load('mlp.params'))  # lemos os parâmetros armazenados no arquivo diretamente.  \
clone.load_state_dict(camadas)  # lemos os parâmetros armazenados no arquivo diretamente.  \
clone.eval()  

MLP(
  (net): Sequential(
    (0): LazyLinear(in_features=0, out_features=50, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=0, out_features=20, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=0, out_features=2, bias=True)
  )
)

In [150]:
clone

MLP(
  (net): Sequential(
    (0): LazyLinear(in_features=0, out_features=50, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=0, out_features=20, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=0, out_features=2, bias=True)
  )
)

In [151]:
[(name, param.shape) for name, param in clone.named_parameters()]

[('net.0.weight', torch.Size([50, 100])),
 ('net.0.bias', torch.Size([50])),
 ('net.2.weight', torch.Size([20, 50])),
 ('net.2.bias', torch.Size([20])),
 ('net.4.weight', torch.Size([2, 20])),
 ('net.4.bias', torch.Size([2]))]

In [152]:
# Posso colocar qualquer entrada X nesta rede:
X = torch.rand(size=(10, 100))
Y = clone(X)                        # Saída Y utilizando como entrada X
Y

tensor([[-1.9255,  0.7424],
        [-1.6536,  0.7815],
        [-2.0473,  0.4035],
        [-1.9340,  0.3080],
        [-1.9688,  0.4775],
        [-1.9776,  0.6307],
        [-1.7683,  0.9331],
        [-1.8743,  0.8061],
        [-1.8371,  0.7640],
        [-1.8530,  0.7020]], grad_fn=<AddmmBackward0>)

--------------------------------------------

# PARTE 3: UTILIZANDO AS CAMADAS APROVEITADAS/CRIADAS EM UMA NOVA REDE

In [168]:
# NOVA REDE:
class NOVA(d2l.Classifier):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
                                 nn.LazyLinear(50),
                                 nn.ReLU(),
                                 nn.Linear(50,20),      # alterado de Lazy para Linear
                                 nn.ReLU6(),            # alterado para ReLU6 
                                 nn.LazyLinear(2)
                                 )

In [169]:
clone = NOVA()                                     # instanciamos um clone do modelo MLP original  \
# clone.load_state_dict(torch.load('mlp.params'))  # lemos os parâmetros armazenados no arquivo diretamente.  \
clone.load_state_dict(camadas)                     # lemos os parâmetros armazenados no arquivo diretamente.  \
clone.eval()  

NOVA(
  (net): Sequential(
    (0): LazyLinear(in_features=0, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU6()
    (4): LazyLinear(in_features=0, out_features=2, bias=True)
  )
)

In [170]:
X = torch.rand(size=(10, 100))
Y = clone(X)                        # Saída Y utilizando a entrada X
Y

tensor([[-1.9299,  0.4467],
        [-1.8426,  0.7746],
        [-1.8470,  0.5628],
        [-1.7955,  0.8536],
        [-1.9411,  0.6458],
        [-1.7669,  0.7647],
        [-1.8334,  0.5955],
        [-1.8755,  0.5875],
        [-1.9239,  0.6542],
        [-1.9461,  0.5672]], grad_fn=<AddmmBackward0>)

# Ex.3 - Como você faria para salvar a arquitetura e os parâmetros da rede? Quais restrições você imporia à arquitetura?

In [240]:
import torch
from torch import nn
from d2l import torch as d2l
from collections import OrderedDict


# CLASSE PARA REDE SEM ENTRADAS (LazyLinear)
class MLP(d2l.Classifier):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
                                 nn.LazyLinear(50),
                                 nn.ReLU(),
                                 nn.LazyLinear(20),
                                 nn.ReLU(),
                                 nn.LazyLinear(2)
                                 )


net = MLP()       # Rede sem entradas
print(f'-->net sem entradas:  {net}\n')


X = torch.rand(size=(10, 100))    # Dados de entrada X
Y = net(X)                        # Saída Y utilizando como entrada X
print(f'-->Y:  {Y}\n')

print(f'-->net com entradas:  {net}\n')

-->net sem entradas:  MLP(
  (net): Sequential(
    (0): LazyLinear(in_features=0, out_features=50, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=0, out_features=20, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=0, out_features=2, bias=True)
  )
)

-->Y:  tensor([[-0.0975,  0.0564],
        [-0.0923,  0.0067],
        [-0.1234,  0.0209],
        [-0.0947,  0.0446],
        [-0.1237,  0.0422],
        [-0.0919,  0.0289],
        [-0.0920,  0.0237],
        [-0.0849,  0.0586],
        [-0.0859,  0.0265],
        [-0.1458,  0.0168]], grad_fn=<AddmmBackward0>)

-->net com entradas:  MLP(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)



In [242]:
# ----------------------------------------------------------------------
# Save:
# torch.save(net.state_dict(), 'mlp.params')          # copiando apenas parâmetros da rede

# Save the model parameters to a separate file
torch.save(net.state_dict(), 'mlp_parameters.pth')   # copiando os parâmetros aprendidos. Extensão: .pth. Vide referências.
# A PTH file is a machine learning model

# Save the model architecture to a script or JSON
torch.save(net, 'mlp_arq.pth')                       # copiando a arquitetura  
# A PTH file is a machine learning model 


# # Load:
# clone = MLP()
# clone.load_state_dict(torch.load('mlp.params'))
# clone.eval()
# print(f'-->clone:  {clone}\n')


net_new = torch.load('mlp_arq.pth')                     # carregando a rede completa
# A PTH file is a machine learning model 

Y_new = net_new(X)
Y_new == Y 

tensor([[True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True]])

In [243]:
net_new.net

Sequential(
  (0): Linear(in_features=100, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=2, bias=True)
)

# Referências:

https://pandalab.me/archives/file_io

https://stackoverflow.com/questions/45114985/how-to-create-an-ordereddict-in-python 

https://stackoverflow.com/questions/24204087/how-to-get-multiple-dictionary-values

https://fileinfo.com/extension/pth